In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,compile, execute,IBMQ
from qiskit import BasicAer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from math import pi
from qiskit.mapper._layout import Layout as Ibmlayout
from qiskit.tools.compiler import compile as qcompile
import matplotlib.pyplot as plt
import sys
from qiskit.tools.visualization import circuit_drawer
from parseRealization import *
%matplotlib inline
testDir="./tests/"

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
#select a backend
IBMQ.load_accounts()
backends = IBMQ.backends(simulator=False,filters=lambda x: x.configuration().n_qubits > 4)
print(backends)
#print(available)
#least_busy = BasicAer.get_backend('qasm_simulator')
least_busy = IBMQ.get_backend('ibmq_16_melbourne')
couplingMap = least_busy.configuration().coupling_map
print(couplingMap)
qubitsSize = least_busy.configuration().n_qubits


[<IBMQBackend('ibmqx4') from IBMQ()>, <IBMQBackend('ibmq_16_melbourne') from IBMQ()>]
[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]


In [3]:
def prepareIBMQLayout(qReg,layout):
    ibmLayout = {}
    for elem in layout:
        print("Element in layout is",elem," physical is",layout[elem])
        quantum = ord(elem)-ord("a")
        physical = ord(layout[elem])-ord("a")
        quantumTuple = (qReg.name, quantum)
        ibmLayout[quantumTuple]=(qReg.name,physical)
    print("tempDictionary",ibmLayout)
    print("items are",ibmLayout.items())
    print("IBMLAYOUT")
    print(ibmLayout)
    return ibmLayout

def bigFunction(fileName):
    ioClass = readCircuitInformation(fileName) 
    answers = ioClass.getKmap()
    size = ioClass.getSize()
    ioClass.readGatesFromFile(fileName)
    backend_sim = BasicAer.get_backend('qasm_simulator')
    error_count = 0
    
    ctg = CircuitTransitionGraph()
    print(ctg.transformCoupling(couplingMap))
    ctg.setSize(size)

    for i in range(0,1):
        #This part corresponds for part of experiment without minimal or no changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        qc,qr = ctg.readGatesFromIOClass(qr,qc, ioClass)
        ctg.layOutQubits()
        tempLayout = ctg.layout.copy()
        ibmLayout = prepareIBMQLayout(qr,tempLayout)
        measureFidelityWithoutChanges(qr,cr,qc,ibmLayout)
        
        #This part corresponds for part of experiment after significant changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        ctg = fixTheStuff(ctg)
        #This one needs to comply with changes you did
        qc,qr = ctg.readFixedGatesFromCtg(qr,qc)
        measureFidelityWithChanges(qr,cr,qc)
    
def fixTheStuff(ctg):
    print("Missing connections are",ctg.getMissingConnections())
    #This one to fix the changes... fixthemissingedges connects stuff around. did not test though
    ctg.fixMissingEdges()
    print("FIxing the stuff")
    return ctg


def measureFidelityWithoutChanges(qr,cr,qc,ibm_layout):
    #least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    qobj = qcompile(qc,least_busy,initial_layout=ibm_layout,pass_manager=None)
    #This line provides print of the compiled circuit qasm
    print(qobj.experiments[0].header.as_dict())#["compiled_circuit_qasm"])
    
    #job = execute(qc,least_busy,shots=200)
    #result = job.result()
    #print(result.get_counts())
    #plot_histogram(result.get_counts())

    
#This needs to be implemented    
def measureFidelityWithChanges(qr,cr,qc):
    least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = execute(qc,least_busy,shots=200)
    result = job.result()
    print(result.get_counts())
    #plot_histogram(result.get_counts())


In [4]:
fileName="ex1"
#if testFromFile(filename) == 0:
fileName = testDir+fileName
bigFunction(fileName)

./tests/ex1.pla
The constants are set to .constants 0-----

The circuit has garbage lines in it or file parsed incorrectly, check ioclass
The circuit has constant inputs in it or file parsed incorrectly, check ioclass
['ab', 'bc', 'cd', 'de', 'ek', 'ef', 'fg', 'fj', 'gi', 'hi', 'ij', 'jk', 'dl', 'kl', 'lm', 'cm', 'bn', 'mn']
Constant field is: 0-----
Length of constants field is: 6
Reached here in preparation of inputs!!!, the inputList is ['0', '0', '0', '0', '0', '1']
SELFLAYOUTOFQUBIT ('a', 5)
Final layout is {'a': 'm', 'c': 'c', 'f': 'l', 'b': 'd', 'd': 'b', 'e': 'a'}
Element in layout is a  physical is m
Element in layout is c  physical is c
Element in layout is f  physical is l
Element in layout is b  physical is d
Element in layout is d  physical is b
Element in layout is e  physical is a
tempDictionary {('q0', 0): ('q0', 12), ('q0', 2): ('q0', 2), ('q0', 5): ('q0', 11), ('q0', 1): ('q0', 3), ('q0', 3): ('q0', 1), ('q0', 4): ('q0', 0)}
items are dict_items([(('q0', 0), ('q0', 12

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning
